In [44]:
from tkinter import *
import pandas as pd 
from tkinter.ttk import *
import re
from textblob import TextBlob
from tqdm.auto import tqdm

# Importar a função que classifica as manchetes
#from funcao.ipynb import classificar
  
# importing askopenfile function 
# from class filedialog 
from tkinter.filedialog import askopenfile
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import f1_score, recall_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score

In [45]:
def cleanup(text):
    import string
    punctuation = '[\/!-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed

In [46]:
data = pd.read_excel("lais.xlsx")   

#------------------------------------------------------------------
manchetes = {}
contador = 0
email=[]
lista_manchetes=[]
numero_atual = 0
loop = tqdm(total = data.shape[0], position = 0, leave = False)
for i in data.HTML.index:
    loop.set_description("Extraindo manchetes...".format(numero_atual))
    loop.update(1)
    numero_atual += 1
    texto = str(data.HTML[i])

    for indice in range(len(texto)):

        if texto[indice:indice+7] == "message":

            manchete_incial = texto[indice+11:indice+150]
            achou = False
            manchete = " "
            indice_letra = 0
            email.append(i+2)

            while indice_letra < len(manchete_incial):

                if manchete_incial[indice_letra] == "}":

                    manchete = manchete_incial[0:indice_letra-6]
                    achou = True
                    manchetes[contador] = manchete

                    manchete_limpa = cleanup(manchete)
                    manchete_min = manchete_limpa.lower()

                    try:
                        hi_blob = TextBlob(manchete_min)
                        manchete_pronta = hi_blob.translate(to='en')
                    except:
                        manchete_pronta = manchete_min

                    lista_manchetes.append(str(manchete_pronta))
                    contador += 1
                    break

                else:

                    indice_letra += 1
loop.close()
#------------------------------------------------------------------
lista_links = []
for linha in data.HTML.index:   #Percorre cada linha do dataset
    texto = data.HTML[linha]

    contador_de_titulos = 0
    for indice in range(len(texto)-7):
        if texto[indice:indice+7] == "message":
            contador_de_titulos += 1



    indice = 0
    texto_com_url = ""

    while indice < len(texto):  #Percorre o texto dentro da linha
        if texto[indice:indice + 7] == "widgets":
            while True:
                if texto[indice+5:indice+8] == "} ]":
                    break
                else:
                    texto_com_url += texto[indice+8]
                    indice += 1
            break 
        else:
            indice += 1


    indice = 0
    numero_atual_de_titulos = 0

    while indice < len(texto_com_url) and numero_atual_de_titulos < contador_de_titulos:
        if texto_com_url[indice : indice+3] == "url":
            link = ""
            while True:
                if texto_com_url[indice+7] == '"':
                    numero_atual_de_titulos += 1
                    break
                else:
                    link += texto_com_url[indice+7]
                    indice += 1
            lista_links.append(link)
        else:
            indice += 1
#------------------------------------------------------------------
indice_lista = 0
while indice_lista < len(lista_links):
    contador_de_https = 0
    contador = 0
    link = lista_links[indice_lista]
    while contador < len(link):
        if link[contador:contador+5] == "https" or link[contador:contador+4] == "http":
            contador_de_https += 1
        if contador_de_https == 2:
            link_completo = link[contador:len(link)]
            link_final = ""
            indice_google = 0
            while indice_google < len(link_completo):

                if link_completo[indice_google : indice_google+7] == "u0026ct":
                    break
                else:
                    link_final += link_completo[indice_google]
                    indice_google += 1

            lista_links[indice_lista] = link_final
            break
        contador += 1

    indice_lista += 1
#------------------------------------------------------------------

dicionario = {}

dicionario['Manchetes'] = lista_manchetes
#dicionario['Indice'] = email
dicionario['Link'] = lista_links

data_manchetes = pd.DataFrame(data=dicionario)

result = pd.concat([data, data_manchetes], ignore_index=True, axis=1)

result.columns = ['Data','De','HTML','Resumo','Manchete','Link']

result["Relevância"] = int(1)

In [47]:
result.head()

,Data,De,HTML,Resumo,Manchete,Link,Relevância
0,2020-04-03 23:46:37,googlealerts-noreply@google.com,"<html lang=""pt-BR""> <head> </head> <body> <div...",Google Bonos del Impacto Social Atualização as...,a lais é legeal,https://www.infobae.com/economia/2020/04/03/bo...,1
1,NaT,NaN,NaN,NaN,la ampliación del bono social costará 23 millo...,https://m.europapress.es/economia/energia-0034...,1
2,NaT,NaN,NaN,NaN,bono de 380 soles beneficiarios podrán retira...,https://larepublica.pe/economia/2020/03/31/bon...,1


In [48]:
treino = pd.read_excel("base_pronta.xlsx")

X_para_treinar = treino.loc[:, "Manchete"]
Y_para_treinar = treino.loc[:, "Relevância"]
##y = arquivo.loc[:, "Relevância"]

# Vetorizando
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_para_treinar).toarray()


# Sepando em teste e treino
# Porém vou colocar tudo para treino
X_train, X_test, y_train, y_test = train_test_split(X_train_counts, Y_para_treinar, test_size=0.01, random_state=42)
#Aplicando o modelo SVM (Support Vector Machine)

kernels = ['linear', 'poly', 'rbf', 'sigmoid', 'precomputed']

model = svm.SVC(kernel = 'linear') #Kernel

#Dando fit no modelo SVM usando o dataset de treino
model.fit(X_train, y_train)

SVC(kernel='linear')

In [49]:
X = result.loc[:, "Manchete"]
pred = count_vect.transform(X).toarray()
predict = model.predict(pred)

In [50]:
predict

array([0, 0, 0])

In [40]:
import numpy as np
lista_prevista = np.array(predict).tolist()

In [41]:
lista_prevista

[0, 0, 0]

In [42]:
result.Manchete = lista_prevista

In [43]:
result

,Data,De,HTML,Resumo,Manchete,Link,Relevância
0,2020-04-03 23:46:37,googlealerts-noreply@google.com,"<html lang=""pt-BR""> <head> </head> <body> <div...",Google Bonos del Impacto Social Atualização as...,0,https://www.infobae.com/economia/2020/04/03/bo...,1
1,NaT,NaN,NaN,NaN,0,https://m.europapress.es/economia/energia-0034...,1
2,NaT,NaN,NaN,NaN,0,https://larepublica.pe/economia/2020/03/31/bon...,1
